#  Ceneo Scraper

##  Components of single opinion 

|Component|Selector|Variable|
|---------|--------|--------|
|opinion ID|["data-entry-id"]|opinion_id|
|opinion's author|span.user-post__author-name|author|
|author's recomendation|span.user-post__author-recomendation > em|recomendation|
|score expressed in number of stars|span.user-post__score-count|score|
|opinion's content|div.user-post__text|content|
|list of products advantages|div.review-feature__title--positives ~ div.review-feature__item|pros|
|list of products disadvantages|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|how many users think that opinion was helpful|span[id^="votes-yes"]|helpful|
|how many users think that opinion was unhelpful|span[id^="votes-no"]|unhelpful|
|publishing date|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|purchase date|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|


## Loading libraries

In [21]:
import json
import os
import requests
from bs4 import BeautifulSoup
from deep_translator import GoogleTranslator

## Function to extract data from HTML code

In [22]:
def extract(ancestor, selector, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute] for tag in ancestor.select(selector)]
        return [tag.get_text().strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try: 
                return ancestor.select_one(selector)[attribute]
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).get_text().strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute]
    return ancestor.get_text().strip()

## Structure of single opinion

In [23]:
selectors={

"opinion_id" : [None,"data-entry-id"],
"author" : ["span.user-post__author-name"],
"recommendation" : ["span.user-post__author-recomendation > em"],
"score" : ["span.user-post__score-count"],
"content" : ["div.user-post__text"],
"pros" : ["div.review-feature__title--positives ~ div.review-feature__item", None, True],
"cons" : ["div.review-feature__title--negatives ~ div.review-feature__item", None, True],
"helpful" : ["button.vote-yes > span"],
"unhelpful" : ["button.vote-no > span"],
"publish_date" : ["span.user-post__published > time:nth-child(1)","datetime"],
"purchase_date" : ["span.user-post__published > time:nth-child(2)","datetime"],
}

## Transformation functions 

In [31]:
def rate(score):
    rate = score.split("/")
    return float(rate[0].replace(",","."))/float(rate[1])
def recommend(recomendation):
    return True if recomendation == "Polecam" else False if recomendation == "Nie polecam" else None 

def translate(text, lang_from = "pl", lang_to = "en"):
    if isinstance(text, list):
        return [GoogleTranslator(source=lang_from, target=lang_to,).translate(t) for t in text]
    return GoogleTranslator(source=lang_from, target=lang_to,).translate(text)


## transformations

In [25]:
transformations = {
"recommendation" : recommend,
"score" : rate,
"helpful" : int,
"unhelpful" : int,
"content" : translate,
"pros" : translate,
"cons" : translate
}

## URL adress for first page with opinions about product 

In [30]:

product_id = input("enter product code: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"



## Extracting all opinions about product from HTML code

In [32]:
all_opinions = []
while(url):
    print(url)
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        for key, value in transformations.items():
            single_opinion[key] = value(single_opinion[key])
        all_opinions.append(single_opinion)
    try:
        url = 'https://www.ceneo.pl'+extract(page_dom, "a.pagination__next", "href")
    except TypeError:
        url = None




https://www.ceneo.pl/64418827#tab=reviews
https://www.ceneo.pl/64418827/opinie-2


## Saving all opinions to JSON file


In [33]:
if not os.path.exists("opinions"):
    os.mkdir("opinions")
jf = open(f"opinions/{product_id}.json", "w", encoding="UTF-8")
json.dump(all_opinions, jf, indent=4, ensure_ascii=False)
jf.close()
